In [1]:
##DEPENDENCIES
import pandas as pd
import datetime
from datetime import timedelta, date
import yfinance as yf
import numpy
import yahoofinancials
from yahoofinancials import YahooFinancials

#define quarter yyyyQq
q = '2020Q2'

In [2]:
#import earnings dates data ***check file***
sp_earnings = pd.read_csv('resources/'+q+'_sp_earnings.csv', index_col=False)
del sp_earnings['Unnamed: 0']
sp_earnings.head()

,Symbol,2020Q2 Time,2020Q2 Date
0,A,After Market Close,2020-05-21
1,AAL,Before Market Open,2020-04-30
2,AAP,Before Market Open,2020-05-20
3,AAPL,After Market Close,2020-04-30
4,ABBV,Before Market Open,2020-05-01


In [3]:
## define function to collect previous, earnings, and next day ohlc quotes given the symbol and earnings date
def ohlc(sym, earn_date):
        yf_sym = YahooFinancials([sym])
        prev = []
        earn = []
        nxt = []
        try:    
            ##PREVIOUS DAY DATA
            #calculating the previous day
            theday = datetime.date(*map(int, earn_date.split('-')))
            prevday = theday - datetime.timedelta(days=1)
            prevday = prevday.strftime('%Y-%m-%d')
            prev_date = prevday
            
            #data for previous day
            data = yf_sym.get_historical_price_data(start_date= prevday, 
                                                  end_date= earn_date, 
                                                  time_interval='daily')
            print(sym+' '+earn_date)
            #if market closed on previous day this keeps trying one day previous until data is captured
            while data[sym] == None:
                theday = datetime.date(*map(int, prevday.split('-')))
                earn_prev = prevday
                prevday = theday - datetime.timedelta(days=1)
                prevday = prevday.strftime('%Y-%m-%d')
                prev_date = prevday
                data = yf_sym.get_historical_price_data(start_date= prevday, 
                                                  end_date= earn_prev, 
                                                  time_interval='daily') 
            #previous day OHLC
            prev_open = data[sym]['prices'][0]['open']
            prev_low = data[sym]['prices'][0]['low']
            prev_high = data[sym]['prices'][0]['high']            
            prev_close = data[sym]['prices'][0]['close']
            prev = [prev_date, prev_open, prev_low, prev_high, prev_close]

            ##EARNINGS DATE DATA
            #calculating next day
            theday = datetime.date(*map(int, earn_date.split('-')))
            nextday = theday + datetime.timedelta(days=1)
            nextday = nextday.strftime('%Y-%m-%d')
            next_date = nextday
            #data for next day
            data = yf_sym.get_historical_price_data(start_date= earn_date, 
                                        end_date= nextday, 
                                        time_interval='daily')
            #if market closed on next day this keeps trying one day ahead until data is captured
            while data[sym] == None:
                theday = datetime.date(*map(int, nextday.split('-')))
                earn_next = nextday
                nextday = theday + datetime.timedelta(days=1)
                nextday = nextday.strftime('%Y-%m-%d')
                data = yf_sym.get_historical_price_data(start_date= earn_next, 
                                                  end_date= nextday, 
                                                  time_interval='daily') 
            #earnings day OHLC
            earn_open = data[sym]['prices'][0]['open']
            earn_low = data[sym]['prices'][0]['low']
            earn_high = data[sym]['prices'][0]['high']            
            earn_close = data[sym]['prices'][0]['close']
            earn = [earn_date, earn_open, earn_low, earn_high, earn_close]
            
            ##NEXT DATE DATA
            #calculating day after next
            theday = datetime.date(*map(int, next_date.split('-')))
            nextday = theday + datetime.timedelta(days=1)
            nextday = nextday.strftime('%Y-%m-%d')
            #data for next day
            data = yf_sym.get_historical_price_data(start_date= next_date, 
                                        end_date= nextday, 
                                        time_interval='daily')
            #if market closed on day after next this keeps trying one day ahead until data is captured
            while data[sym] == None:
                theday = datetime.date(*map(int, nextday.split('-')))
                earn_next = nextday
                next_date = nextday
                nextday = theday + datetime.timedelta(days=1)
                nextday = nextday.strftime('%Y-%m-%d')
                data = yf_sym.get_historical_price_data(start_date= earn_next, 
                                                  end_date= nextday, 
                                                  time_interval='daily') 
            #next day OHLC
            next_open = data[sym]['prices'][0]['open']
            next_low = data[sym]['prices'][0]['low']
            next_high = data[sym]['prices'][0]['high']            
            next_close = data[sym]['prices'][0]['close']
            nxt.append(next_date) 
            nxt.append(next_open)
            nxt.append(next_low)
            nxt.append(next_high)
            nxt.append(next_close)
            print(sym+' '+earn_date)
        except:
            print(sym+' '+str(earn_date))
            prev_date = 'NaN'
            prev_open = 'NaN'
            prev_low = 'NaN'
            prev_high = 'NaN'            
            prev_close = 'NaN'
            #earn_date = 'NaN'
            earn_open = 'NaN'
            earn_low = 'NaN'
            earn_high = 'NaN'       
            earn_close = 'NaN'
            next_date = 'NaN'
            next_open = 'NaN'
            next_low = 'NaN'
            next_high = 'NaN'       
            next_close = 'NaN' 

        quotes = {'Symbol' : sym,
                q+' Prev Date' : prev_date,
                q+' Prev Open' : prev_open, 
                q+' Prev High' : prev_high, 
                q+' Prev Low' : prev_low, 
                q+' Prev Close' : prev_close, 
                q+' Earn Date' : earn_date, 
                q+' Earn Open' : earn_open, 
                q+' Earn High' : earn_high, 
                q+' Earn Low' : earn_low, 
                q+' Earn Close' : earn_close,
                q+' Next Date' : next_date, 
                q+' Next Open' : next_open,
                q+' Next High' : next_high,
                q+' Next Low' : next_low, 
                q+' Next Close' : next_close}
        return quotes;

In [4]:
#collect ohlc data for every symbol and add each dictionary to a list
symbols = list(sp_earnings['Symbol'])
date_prices = []

for row in sp_earnings.iterrows():
    quotes = ohlc(row[1]['Symbol'], row[1][q+' Date'])
    date_prices.append(quotes)

date_prices    

A 2020-05-21
A 2020-05-21
AAL 2020-04-30
AAL 2020-04-30
AAP 2020-05-20
AAP 2020-05-20
AAPL 2020-04-30
AAPL 2020-04-30
ABBV 2020-05-01
ABBV 2020-05-01
ABC 2020-05-07
ABC 2020-05-07
ABMD 2020-04-30
ABMD 2020-04-30
ABT 2020-04-16
ABT 2020-04-16
ACN 2020-06-25
ACN 2020-06-25
ADBE 2020-06-11
ADBE 2020-06-11
ADI 2020-05-20
ADI 2020-05-20
ADM 2020-04-29
ADM 2020-04-29
ADP 2020-04-29
ADP 2020-04-29
ADS 2020-04-23
ADS 2020-04-23
ADSK 2020-05-21
ADSK 2020-05-21
AEE 2020-05-11
AEE 2020-05-11
AEP 2020-05-06
AEP 2020-05-06
AES 2020-05-07
AES 2020-05-07
AFL 2020-04-29
AFL 2020-04-29
AGN 2020-05-05
AGN 2020-05-05
AIG 2020-05-04
AIG 2020-05-04
AIV 2020-05-07
AIV 2020-05-07
AIZ nan
AJG 2020-04-30
AJG 2020-04-30
AKAM 2020-04-28
AKAM 2020-04-28
ALB 2020-05-06
ALB 2020-05-06
ALGN 2020-04-29
ALGN 2020-04-29
ALK 2020-05-05
ALK 2020-05-05
ALL 2020-05-05
ALL 2020-05-05
ALLE 2020-04-23
ALLE 2020-04-23
ALXN 2020-05-06
AMAT 2020-05-14
AMCR nan
AMD 2020-04-28
AMD 2020-04-28
AME 2020-05-05
AME 2020-05-05
AMGN 2020

LUV 2020-04-28
LUV 2020-04-28
LVS 2020-04-22
LVS 2020-04-22
LW 2020-04-01
LW 2020-04-01
LYB 2020-05-01
LYB 2020-05-01
LYV 2020-04-30
LYV 2020-04-30
MA 2020-04-29
MA 2020-04-29
MAA 2020-05-06
MAA 2020-05-06
MAR 2020-05-08
MAR 2020-05-08
MAS 2020-04-29
MAS 2020-04-29
MCD 2020-04-30
MCD 2020-04-30
MCHP nan
MCK 2020-05-20
MCK 2020-05-20
MCO 2020-04-30
MCO 2020-04-30
MDLZ 2020-04-28
MDLZ 2020-04-28
MDT 2020-05-21
MDT 2020-05-21
MET nan
MGM 2020-04-30
MGM 2020-04-30
MHK 2020-05-04
MHK 2020-05-04
MKC 2020-06-25
MKC 2020-06-25
MKTX 2020-04-29
MKTX 2020-04-29
MLM 2020-04-28
MLM 2020-04-28
MMC 2020-04-30
MMC 2020-04-30
MMM 2020-04-28
MMM 2020-04-28
MNST 2020-04-30
MNST 2020-04-30
MO 2020-04-30
MO 2020-04-30
MOS Multi
MPC 2020-05-05
MPC 2020-05-05
MRK 2020-04-28
MRK 2020-04-28
MRO 2020-05-06
MRO 2020-05-06
MS 2020-04-16
MS 2020-04-16
MSCI 2020-04-28
MSCI 2020-04-28
MSFT 2020-04-29
MSFT 2020-04-29
MSI nan
MTB 2020-04-20
MTB 2020-04-20
MTD nan
MU 2020-06-23
MU 2020-06-23
MXIM 2020-04-28
MXIM 2020-0

[{'Symbol': 'A',
  '2020Q2 Prev Date': 'NaN',
  '2020Q2 Prev Open': 'NaN',
  '2020Q2 Prev High': 'NaN',
  '2020Q2 Prev Low': 'NaN',
  '2020Q2 Prev Close': 'NaN',
  '2020Q2 Earn Date': '2020-05-21',
  '2020Q2 Earn Open': 'NaN',
  '2020Q2 Earn High': 'NaN',
  '2020Q2 Earn Low': 'NaN',
  '2020Q2 Earn Close': 'NaN',
  '2020Q2 Next Date': 'NaN',
  '2020Q2 Next Open': 'NaN',
  '2020Q2 Next High': 'NaN',
  '2020Q2 Next Low': 'NaN',
  '2020Q2 Next Close': 'NaN'},
 {'Symbol': 'AAL',
  '2020Q2 Prev Date': 'NaN',
  '2020Q2 Prev Open': 'NaN',
  '2020Q2 Prev High': 'NaN',
  '2020Q2 Prev Low': 'NaN',
  '2020Q2 Prev Close': 'NaN',
  '2020Q2 Earn Date': '2020-04-30',
  '2020Q2 Earn Open': 'NaN',
  '2020Q2 Earn High': 'NaN',
  '2020Q2 Earn Low': 'NaN',
  '2020Q2 Earn Close': 'NaN',
  '2020Q2 Next Date': 'NaN',
  '2020Q2 Next Open': 'NaN',
  '2020Q2 Next High': 'NaN',
  '2020Q2 Next Low': 'NaN',
  '2020Q2 Next Close': 'NaN'},
 {'Symbol': 'AAP',
  '2020Q2 Prev Date': 'NaN',
  '2020Q2 Prev Open': 'NaN',
 

In [5]:
#create a dataframe from the ohlc list
dp_df = pd.DataFrame(date_prices)
dp_df = dp_df[['Symbol',
  q+' Prev Date',
  q+' Prev Open',
  q+' Prev High',
  q+' Prev Low',
  q+' Prev Close',
  q+' Earn Date',
  q+' Earn Open',
  q+' Earn High',
  q+' Earn Low',
  q+' Earn Close',
  q+' Next Date',
  q+' Next Open',
  q+' Next High',
  q+' Next Low',
  q+' Next Close']]
dp_df.head()

,Symbol,2020Q2 Prev Date,2020Q2 Prev Open,2020Q2 Prev High,2020Q2 Prev Low,2020Q2 Prev Close,2020Q2 Earn Date,2020Q2 Earn Open,2020Q2 Earn High,2020Q2 Earn Low,2020Q2 Earn Close,2020Q2 Next Date,2020Q2 Next Open,2020Q2 Next High,2020Q2 Next Low,2020Q2 Next Close
0,A,NaN,NaN,NaN,NaN,NaN,2020-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAL,NaN,NaN,NaN,NaN,NaN,2020-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAP,NaN,NaN,NaN,NaN,NaN,2020-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,NaN,NaN,NaN,NaN,NaN,2020-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBV,NaN,NaN,NaN,NaN,NaN,2020-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#calculate the percent changes following earnings and added to dataframe
bopen_change = []
aclose_change = []
for row in dp_df.iterrows():
    pr_cl = float(row[1][q+' Prev Close'])
    e_op = float(row[1][q+' Earn Open'])
    e_cl = float(row[1][q+' Earn Close'])
    nx_op = float(row[1][q+' Next Open'])
    bo_change = ((e_op - pr_cl)/(pr_cl))*100
    bo_change = round(bo_change, 2)
    ac_change = ((nx_op - e_cl)/(e_cl))*100
    ac_change = round(ac_change, 2)
    bopen_change.append(bo_change)
    aclose_change.append(ac_change)

dp_df[q+' Before Open %Change'] = bopen_change
dp_df[q+' After Close %Change'] = aclose_change
dp_df


,Symbol,2020Q2 Prev Date,2020Q2 Prev Open,2020Q2 Prev High,2020Q2 Prev Low,2020Q2 Prev Close,2020Q2 Earn Date,2020Q2 Earn Open,2020Q2 Earn High,2020Q2 Earn Low,2020Q2 Earn Close,2020Q2 Next Date,2020Q2 Next Open,2020Q2 Next High,2020Q2 Next Low,2020Q2 Next Close,2020Q2 Before Open %Change,2020Q2 After Close %Change
0,A,NaN,NaN,NaN,NaN,NaN,2020-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAL,NaN,NaN,NaN,NaN,NaN,2020-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAP,NaN,NaN,NaN,NaN,NaN,2020-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,NaN,NaN,NaN,NaN,NaN,2020-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBV,NaN,NaN,NaN,NaN,NaN,2020-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ABC,NaN,NaN,NaN,NaN,NaN,2020-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ABMD,NaN,NaN,NaN,NaN,NaN,2020-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ABT,2020-04-15,90.01,91.9,88.51,90.94,2020-04-16,92.6,96.99,91.14,96,2020-04-17,97.45,97.75,94.24,96.01,1.83,1.51
8,ACN,NaN,NaN,NaN,NaN,NaN,2020-06-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ADBE,NaN,NaN,NaN,NaN,NaN,2020-06-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#update sp_earnings.csv file
sp_merged = pd.merge(sp_earnings, dp_df, how='outer', on=['Symbol'])
sp_merged.to_csv('resources/'+q+'_sp_earnings.csv')
sp_merged.head()

,Symbol,2020Q2 Time,2020Q2 Date,2020Q2 Prev Date,2020Q2 Prev Open,2020Q2 Prev High,2020Q2 Prev Low,2020Q2 Prev Close,2020Q2 Earn Date,2020Q2 Earn Open,2020Q2 Earn High,2020Q2 Earn Low,2020Q2 Earn Close,2020Q2 Next Date,2020Q2 Next Open,2020Q2 Next High,2020Q2 Next Low,2020Q2 Next Close,2020Q2 Before Open %Change,2020Q2 After Close %Change
0,A,After Market Close,2020-05-21,NaN,NaN,NaN,NaN,NaN,2020-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAL,Before Market Open,2020-04-30,NaN,NaN,NaN,NaN,NaN,2020-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAP,Before Market Open,2020-05-20,NaN,NaN,NaN,NaN,NaN,2020-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAPL,After Market Close,2020-04-30,NaN,NaN,NaN,NaN,NaN,2020-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBV,Before Market Open,2020-05-01,NaN,NaN,NaN,NaN,NaN,2020-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
